In this example the goal is to estimate hidden states of a Dynamical process where all hidden states are Gaussians. Our process can be described with the following equations:

$$
\begin{aligned}
 p(\tau) & = \text{GammaShapeRate}(\alpha_{1}, \beta_{1}),\\
 p(\theta) & = \text{GammaShapeRate}(\alpha_{2}, \beta_{2}),\\
 p(z_i|z_{i - 1}) & = \mathcal{N}(x_i| z_{i - 1} + 1, \tau),\\
 p(y_i|z_i) & = \mathcal{N}(y_i|(z_i - s)^2, \theta),
\end{aligned}
$$

where $z_i$ are hidden states, $y_i$ are noisy nonlinear observations, $\tau$, and $\theta$ are priors for state transition noise and observation noise precision, respectively.

To model this process in `ReactiveMP`, first, we start with importing all needed packages:

In [1]:
using ReactiveMP, Rocket, GraphPPL, Random, LinearAlgebra, Plots, Flux, ForwardDiff, Plots

┌ Info: Precompiling ReactiveMP [a194aa59-28ba-4574-a09c-4a745416d6e3]
└ @ Base loading.jl:1664


Next step, is to generate some synthetic data:

In [ ]:
seed = 123

rng = MersenneTwister(seed)

P = 5
sensor_var = 5

# For large `n` apply: smoothing(model_options(limit_stack_depth = 500), ...)
T = 50

sensor_location = 53

hidden = collect(1:T)
data   = (hidden + rand(rng, NormalMeanVariance(0.0, sqrt(P)), T));
transformed = (data .- sensor_location).^2 + rand(rng, NormalMeanVariance(0.0, sensor_var), T);

Let's plot our synthetic dataset. 
The blue line represents our hidden state we want to estimate using noisy observations, 
which are represented as dots.

In [ ]:
px = plot()

px = plot!(px, hidden, label = "Hidden Signal", color = :blue)
px = scatter!(px, transformed, label = "Non linear observations", markersize = 2, color = :orange)

plot(px)

To create a model we use `GraphPPL` package and `@model` macro:

In [ ]:
function f(z)
    (z - sensor_location)^2
end

@model function sensor_based_observation(T, n_iterations, n_samples, learning_rate)

    z = randomvar(T)
    x = randomvar(T)
    y = datavar(Float64, T)

    τ ~ GammaShapeRate(1.0, 1.0e-12)
    θ ~ GammaShapeRate(1.0, 1.0e-12)
    
    z[1] ~ NormalMeanPrecision(0, τ)
    x[1] ~ f(z[1]) where {meta = CVIApproximation(n_iterations, n_samples, Descent(learning_rate))}
    y[1] ~ NormalMeanPrecision(x[1], θ)

    for t in 2:T
        z[t] ~ NormalMeanPrecision(z[t-1] + 1, τ)
        x[t] ~ f(z[t]) where {meta = CVIApproximation(n_iterations, n_samples, Descent(learning_rate))}
        y[t] ~ NormalMeanPrecision(x[t], θ)
    end

    return z, x, y
end

constraints = @constraints begin
    q(z, x, τ, θ) = q(z)q(x)q(τ)q(θ)
end;

To run inference we will use `inference` function from `ReactiveMP`.
Note, that for running inference for this model we need to init marginals for z, $\theta$, $\tau$, and init z messages.

In [ ]:
res = inference(
    model = Model(sensor_based_observation, T, 1000, 2000, 0.1),
    data = (y = transformed,),
    iterations = 100,
    free_energy = false,
    returnvars = (z = KeepLast(),),
    constraints = constraints,
    initmessages = (z = NormalMeanVariance(0, P),),
    initmarginals = (z = NormalMeanVariance(0, P), τ = GammaShapeRate(1.0, 1.0e-12), θ = GammaShapeRate(1.0, 1.0e-12),),
)

In [ ]:
px = plot()

px = plot!(px,  hidden, label = "Hidden Signal", color = :red)
px = plot!(px,  map(mean, res.posteriors[:z]), label = "Estimated signal location", color = :orange)
px = plot!(px,  map(mean, res.posteriors[:z]), ribbon = std.(9 .* res.posteriors[:z]).|> sqrt, fillalpha = 0.5, label = "Estimated Signal confidence", color = :blue)


plot(px)